
# STAT 3210 Project Milestone – Option B: Data Exploration and Modeling  
**Author:** Bing Huang  

This notebook explores housing prices in Portland, Oregon, using a publicly available dataset (e.g., from Redfin or a similar housing data source).  

We will:
1. Load and inspect the dataset  
2. Perform exploratory data analysis (EDA)  
3. Fit baseline and advanced regression models  
4. Briefly evaluate model performance  


## 1. Setup and Imports

In [ ]:

# If needed, install any missing packages (uncomment as necessary)
# !pip install pandas numpy matplotlib seaborn scikit-learn

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Plot styling
sns.set(style="whitegrid")
pd.set_option("display.max_columns", None)


## 2. Load the Dataset

In [ ]:

# TODO: Replace this path/URL with your actual dataset location
# Example options:
# - A local CSV file: 'data/portland_housing.csv'
# - A direct URL to a CSV export from Redfin or Kaggle

data_path = "portland_housing.csv"  # <-- update this

try:
    df = pd.read_csv(data_path)
    print("Data loaded successfully!")
except FileNotFoundError:
    print("File not found. Please check `data_path` and ensure the CSV exists.")
    df = pd.DataFrame()  # placeholder so later cells don't error immediately

df.head()


## 3. Basic Data Inspection

In [ ]:

# Shape of the dataset
print("Number of rows:", df.shape[0])
print("Number of columns:", df.shape[1])

# Column names
print("\nColumns:")
print(df.columns.tolist())

# Data types and non-null counts
print("\nInfo:")
df.info()


In [ ]:

# Basic descriptive statistics for numeric columns
df.describe().T


## 4. Missing Values

In [ ]:

# Count missing values per column
missing_counts = df.isnull().sum().sort_values(ascending=False)
missing_counts


In [ ]:

# Visualize missingness (optional, simple bar plot)
missing_counts = df.isnull().sum()
missing_counts = missing_counts[missing_counts > 0]

if not missing_counts.empty:
    plt.figure(figsize=(8, 4))
    missing_counts.sort_values(ascending=False).plot(kind="bar")
    plt.title("Missing Values per Feature")
    plt.ylabel("Count of Missing Values")
    plt.tight_layout()
    plt.show()
else:
    print("No missing values detected in the dataset.")


## 5. Exploratory Data Analysis (EDA)

In [ ]:

# Select some key numeric features for quick exploration
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
numeric_cols


In [ ]:

# Histograms for numeric features
if not df.empty and len(numeric_cols) > 0:
    df[numeric_cols].hist(bins=30, figsize=(12, 10))
    plt.suptitle("Histograms of Numeric Features", y=1.02)
    plt.tight_layout()
    plt.show()
else:
    print("No numeric columns found or dataframe is empty.")


In [ ]:

# Example: Scatterplot of square footage vs sale price (update column names as needed)
# Change 'sqft' and 'price' below to match your actual dataset columns.
sqft_col = "square_feet"   # e.g., 'SqFt', 'TOTAL_LIV_AREA', etc.
price_col = "price"        # e.g., 'SalePrice', 'PRICE', etc.

if sqft_col in df.columns and price_col in df.columns:
    plt.figure(figsize=(6, 4))
    sns.scatterplot(data=df, x=sqft_col, y=price_col, alpha=0.5)
    plt.title(f"{sqft_col} vs {price_col}")
    plt.tight_layout()
    plt.show()
else:
    print(f"Please update `sqft_col` and `price_col` to match your dataset columns. Current columns: {df.columns.tolist()}")


In [ ]:

# Correlation matrix & heatmap for numeric features
if not df.empty and len(numeric_cols) > 1:
    corr = df[numeric_cols].corr()
    plt.figure(figsize=(10, 8))
    sns.heatmap(corr, annot=False, cmap="coolwarm", center=0)
    plt.title("Correlation Heatmap (Numeric Features)")
    plt.tight_layout()
    plt.show()
else:
    print("Not enough numeric columns for a correlation heatmap.")


## 6. Data Preparation for Modeling

In [ ]:

# TODO: Adjust feature/target column names based on your dataset
target_col = "price"  # e.g., 'SalePrice' or similar

# Example set of candidate features (update these)
feature_cols = [
    "square_feet",
    "bedrooms",
    "bathrooms",
    "year_built",
    "lot_size"
]

# Keep only columns that actually exist in the dataframe
feature_cols = [col for col in feature_cols if col in df.columns]

print("Using features:", feature_cols)
print("Target:", target_col)

# Drop rows with missing values in selected columns (simple strategy for milestone)
model_df = df[feature_cols + [target_col]].dropna()

X = model_df[feature_cols]
y = model_df[target_col]

print("Model dataset shape:", X.shape)


In [ ]:

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)


## 7. Baseline Model – Linear Regression

In [ ]:

lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

y_pred_lr = lin_reg.predict(X_test)

mse_lr = mean_squared_error(y_test, y_pred_lr)
rmse_lr = np.sqrt(mse_lr)
r2_lr = r2_score(y_test, y_pred_lr)

print("Linear Regression Performance:")
print(f"  RMSE: {rmse_lr:.2f}")
print(f"  R^2 : {r2_lr:.3f}")


## 8. Advanced Models – Random Forest and Gradient Boosting

In [ ]:

# Random Forest Regressor
rf = RandomForestRegressor(
    n_estimators=200,
    random_state=42,
    n_jobs=-1
)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

mse_rf = mean_squared_error(y_test, y_pred_rf)
rmse_rf = np.sqrt(mse_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print("Random Forest Performance:")
print(f"  RMSE: {rmse_rf:.2f}")
print(f"  R^2 : {r2_rf:.3f}")


In [ ]:

# Gradient Boosting Regressor
gbr = GradientBoostingRegressor(
    n_estimators=200,
    learning_rate=0.05,
    random_state=42
)
gbr.fit(X_train, y_train)
y_pred_gbr = gbr.predict(X_test)

mse_gbr = mean_squared_error(y_test, y_pred_gbr)
rmse_gbr = np.sqrt(mse_gbr)
r2_gbr = r2_score(y_test, y_pred_gbr)

print("Gradient Boosting Performance:")
print(f"  RMSE: {rmse_gbr:.2f}")
print(f"  R^2 : {r2_gbr:.3f}")


## 9. Summary and Next Steps


**Summary (Milestone):**  
- Loaded the Portland housing dataset and inspected its structure  
- Performed basic EDA with descriptive statistics and visualizations  
- Trained a baseline Linear Regression model  
- Trained more flexible models (Random Forest and Gradient Boosting) and compared performance  

**Next steps:**  
- Improve feature engineering (e.g., interaction terms, log-transforming price or square footage)  
- Incorporate additional neighborhood-level data if available  
- Tune hyperparameters for the ensemble models  
- Create clearer visualizations to communicate findings in the final project report and presentation  
